In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Dropout, Embedding
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.utils as ku
from sklearn.model_selection import train_test_split

import string
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('cumpa_lyrics.csv')
songs = df[['lyric']]
songs.loc[:,'lyric'] = songs.loc[:,'lyric'].str.lower()
songs['lyric'] = songs['lyric'].str.replace(f'[{string.punctuation}]','')

In [30]:
def secuenciador(df,columna= 'lyric', n =None):
  """
    devuelve np.array de la secuencia de entradas, np.array de las etiquetas de salida y el numero total de palabras existentes en el df
    toma como parametro la columna del df de la que se extraen las letras y el numero que representa la longuitud que van a tener los datos de entrada
    en caso que el n sea None o mayor que el maximo-1 se tomara el maximo -1
  """
  lista_canciones = list(df[columna])
  tokenaizer = Tokenizer()
  tokenaizer.fit_on_texts(lista_canciones)
  numero_palabras = len(tokenaizer.index_word) + 1

  #con el tokenaizer se convierten las palabras a numeras y luego por parrafo toma la primer palabra va agregando de a una por item de la lista hasta completar el parrafo
  secuencia_entrada = []
  for l in lista_canciones:
      tokenlist = tokenaizer.texts_to_sequences([l])[0]
      for i in range(1, len(tokenlist)):
          ngram_sequence = tokenlist[:i+1]
          secuencia_entrada.append(ngram_sequence)

  #Saca el maximo de palabras en cada item y usa ese maximo para rellenar el resto con 0 asi todas tienen el mismo largo
  max_secuencia_entrada = max([len(x) for x in secuencia_entrada])
  if n==None or n > max_secuencia_entrada-1:
    n = max_secuencia_entrada -1
  secuencia_entrada = np.array(pad_sequences(secuencia_entrada, maxlen=n, padding='pre'))

  #separa la x de la y para poder ingresar al modelo
  predictores_modelo, etiquetas_salida = secuencia_entrada[:,:-1], secuencia_entrada[:,-1]
  etiquetas_salida = ku.to_categorical(etiquetas_salida, num_classes=numero_palabras)

  return predictores_modelo, etiquetas_salida, numero_palabras

In [47]:
modelo = Sequential()
modelo.add(Embedding(numero_palabras, 140, input_length=15))
modelo.add(Bidirectional(LSTM(200, return_sequences=True)))
modelo.add(Dropout(0.2))
modelo.add(Bidirectional(LSTM(100, return_sequences=True)))
modelo.add(Dropout(0.2))
modelo.add(LSTM(100, return_sequences=False))
modelo.add(Dense(numero_palabras/2, activation = 'relu', kernel_regularizer=regularizers.l2(0.001)))
modelo.add(Dense(numero_palabras, activation='softmax'))

modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15, 140)           629300    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 15, 400)           545600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 400)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 15, 200)           400800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 15, 200)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_4 (Dense)              (None, 2247)             

In [37]:
predictores_modelo, etiquetas_salida, numero_palabras= secuenciador(songs, columna='lyric', n=16)

In [38]:
x_train, x_test, y_train, y_test = train_test_split(predictores_modelo, etiquetas_salida, shuffle= False, test_size= 0.2)

In [48]:
history = modelo.fit(x_train, y_train, batch_size=128,validation_data=(x_test,y_test), epochs=50, verbose=1)

Epoch 1/50
222/222 [==============================] - 6s 29ms/step - loss: 6.7117 - accuracy: 0.0313 - val_loss: 6.8919 - val_accuracy: 0.0249
Epoch 2/50
222/222 [==============================] - 5s 23ms/step - loss: 6.2168 - accuracy: 0.0356 - val_loss: 6.9332 - val_accuracy: 0.0320
Epoch 3/50
222/222 [==============================] - 5s 23ms/step - loss: 5.9444 - accuracy: 0.0486 - val_loss: 7.0499 - val_accuracy: 0.0502
Epoch 4/50
222/222 [==============================] - 5s 23ms/step - loss: 5.7178 - accuracy: 0.0667 - val_loss: 7.1621 - val_accuracy: 0.0700
Epoch 5/50
222/222 [==============================] - 5s 23ms/step - loss: 5.5157 - accuracy: 0.0809 - val_loss: 7.1763 - val_accuracy: 0.0712
Epoch 6/50
222/222 [==============================] - 5s 23ms/step - loss: 5.3576 - accuracy: 0.0883 - val_loss: 7.2380 - val_accuracy: 0.0744
Epoch 7/50
222/222 [==============================] - 5s 23ms/step - loss: 5.2330 - accuracy: 0.0980 - val_loss: 7.2484 - val_accuracy: 0.0733

In [52]:
def generador_Canciones(text, cantidad_palabras,n):
    for i in range(cantidad_palabras):
        tokenlist = tokenaizer.texts_to_sequences([text])[0]
        tokenlist = pad_sequences([tokenlist], maxlen= n)
        palabra_index = modelo.predict_classes(tokenlist, verbose=0)
        for p, i in tokenaizer.word_index.items():
            if i == palabra_index:
                text =text + ' ' + p
                break
    return text

In [53]:
letra = generador_Canciones('este faso', 20,15)

In [54]:
letra

'este faso o lo que usted me gusta lo que hablo de mi lado siempre estar que me viniste re chuponeada re'